<a href="https://colab.research.google.com/github/nickLin1225/Final-Psy-LineBot/blob/main/final_psy_LineBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install line-bot-sdk flask
!pip install pyngrok
!ngrok authtoken 2QrnEFtjjMkbdQs609UXhiHh7JP_41yYQ94u84CsaxFZoiaJ5
!git clone https://github.com/THUDM/ChatGLM2-6B

In [ ]:
# 添加peft、opencc
!pip install -r /content/ChatGLM2-6B/requirements.txt

In [ ]:
!unzip chatGLM2_6B_QLoRA_t16_v5-20230729T023644Z-001.zip

In [ ]:
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch
base_model_path = "THUDM/chatglm2-6b"
peft_model_path = "/content/chatGLM2_6B_QLoRA_t16_v5/checkpoint-1700"

config = PeftConfig.from_pretrained(peft_model_path)


tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
base_model = AutoModel.from_pretrained(base_model_path,
                      trust_remote_code=True,
                      device='cuda')

model = PeftModel.from_pretrained(base_model, peft_model_path)

In [3]:
!pip install opencc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.3/778.3 kB 6.2 MB/s eta 0:00:00


In [4]:
from flask import Flask, request, abort
from linebot.models.events import FollowEvent
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import requests
from pyngrok import ngrok
import json
import opencc
import csv

# 建置主程序
app = Flask(__name__)
# Channel access token
line_bot_api = LineBotApi("lDOvYWU5DjtPpSVFrNKfNAejGKKSZvKYvhVMBR1ND1zmh8Got+JtoF4EgpWRconwzhxkKWWpwbIsby4+vLETeg+dMhUU8M6mL+7dmNFnHl1QbtdT66wv3gIlj90GKuECO7r/q/wVPkkNnCGtnsfRKAdB04t89/1O/w1cDnyilFU=")
# Channel secret
handler = WebhookHandler("f8f7fe98ab2ebbf5c4af4ace91a7b7f1")


# 啟動server對外接口，使line能夠丟消息進來
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

<ipython-input-4-1514612bc7b9>:15: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi("lDOvYWU5DjtPpSVFrNKfNAejGKKSZvKYvhVMBR1ND1zmh8Got+JtoF4EgpWRconwzhxkKWWpwbIsby4+vLETeg+dMhUU8M6mL+7dmNFnHl1QbtdT66wv3gIlj90GKuECO7r/q/wVPkkNnCGtnsfRKAdB04t89/1O/w1cDnyilFU=")
<ipython-input-4-1514612bc7b9>:17: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler("f8f7fe98ab2ebbf5c4af4ace91a7b7f1")


In [ ]:
'''
回應用戶，歡迎用的文字消息

'''

@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('哈囉，歡迎來到心理聊天室，有什麼問題都可以跟我說呦！')]
    )

In [ ]:
'''
爬取心理文章

'''

import requests
from bs4 import BeautifulSoup
import csv
import random

# 要爬取的網頁 URL
url = "https://www.iiispace.com/blog/"

# 發送 HTTP GET 請求並獲取網頁內容
response = requests.get(url)
html_content = response.text

# 使用 BeautifulSoup 解析 HTML 內容
soup = BeautifulSoup(html_content, "html.parser")

# 找到所有文章標題和連結
articles = soup.find_all("h2")
data = []

# 爬取文章資訊
for article in articles[:20]:
    title = article.a.text
    link = article.a["href"]
    article_response = requests.get(link)
    article_html = article_response.text
    article_soup = BeautifulSoup(article_html, "html.parser")
    image = article_soup.find("img", class_="enigma_img_responsive wp-post-image")
    image_url = image["src"] if image else "沒有圖片"
    data.append((title, link, image_url))

# 隨機選取 5 筆文章
random_sample = random.sample(data, 5)
carousel_columns = []

# 建立 CarouselColumns
for title, link, image_url in random_sample:
    carousel_column = CarouselColumn(
        thumbnail_image_url=image_url,
        title="近期的心理文章",
        text=title,
        actions=[
            URIAction(label="詳細內容", uri=link)
        ]
    )
    carousel_columns.append(carousel_column)

carousel_template = CarouselTemplate(columns=carousel_columns)

# Message event
@handler.add(MessageEvent)
def handle_message(event):
    reply_token = event.reply_token
    if event.message.text == '近期的心理文章':
        get_psy_info(event)
    else:
        line_bot_api.reply_message(reply_token, TextSendMessage("請重新輸入"))

# 回覆 Carousel Template 訊息
def get_psy_info(event):
    random_sample = random.sample(data, 5)
    carousel_columns = []

    for title, link, image_url in random_sample:
        carousel_column = CarouselColumn(
            thumbnail_image_url=image_url,
            title="近期的心理文章",
            text=title,
            actions=[
                URIAction(label="詳細內容", uri=link)
            ]
        )
        carousel_columns.append(carousel_column)

    carousel_template = CarouselTemplate(columns=carousel_columns)

    line_bot_api.reply_message(
        event.reply_token,
        TemplateSendMessage(alt_text="心理相關諮訊", template=carousel_template)
    )

In [9]:
from linebot.exceptions import LineBotApiError

# 創建 Rich Menu
rich_menu_to_create = RichMenu(
    size=RichMenuSize(width=2500, height=843),
    selected=True,
    name="My Rich Menu",
    chat_bar_text="查看更多資訊",
    areas=[
        RichMenuArea(
            bounds=RichMenuBounds(x=0, y=0, width=833, height=843),
            action=URIAction(label='Button 1', uri='https://forms.gle/7jbBMRh1CaphF8hv7')
        ),
        RichMenuArea(
            bounds=RichMenuBounds(x=833, y=0, width=833, height=843),
            action=URIAction(label='Button 2', uri='https://www.iiispace.com/blog/')
        ),
        RichMenuArea(
            bounds=RichMenuBounds(x=1666, y=0, width=833, height=843),
            action=MessageAction(label='Button 3', text='近期的心理文章')
        )
    ]
)

# 上傳 Rich Menu
try:
    rich_menu_id = line_bot_api.create_rich_menu(rich_menu=rich_menu_to_create)
    print("Rich Menu created. ID:", rich_menu_id)
except LineBotApiError as e:
    print("Failed to create Rich Menu:", e)

# 上傳 Rich Menu 的圖片
image_url = 'https://i.imgur.com/eHvGozp.jpg'
image_content = requests.get(image_url).content
line_bot_api.set_rich_menu_image(rich_menu_id, 'image/png', image_content)

# 設定預設 Rich Menu
line_bot_api.set_default_rich_menu(rich_menu_id)

# 獲取現有的 Rich Menu 列表
# rich_menu_list = line_bot_api.get_rich_menu_list()

# 刪除現有的 Rich Menu
# for rich_menu in rich_menu_list:
#     line_bot_api.delete_rich_menu(rich_menu.rich_menu_id)

<ipython-input-9-00df1af999a2>:27: LineBotSdkDeprecatedIn30: Call to deprecated method create_rich_menu. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).create_rich_menu(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  rich_menu_id = line_bot_api.create_rich_menu(rich_menu=rich_menu_to_create)


Rich Menu created. ID: richmenu-7c5fb911244c8fe06d143db2120ce068


<ipython-input-9-00df1af999a2>:35: LineBotSdkDeprecatedIn30: Call to deprecated method set_rich_menu_image. (Use 'from linebot.v3.messaging import MessagingApiBlob' and 'MessagingApiBlob.set_rich_menu_image(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.set_rich_menu_image(rich_menu_id, 'image/png', image_content)
<ipython-input-9-00df1af999a2>:38: LineBotSdkDeprecatedIn30: Call to deprecated method set_default_rich_menu. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_default_rich_menu(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.set_default_rich_menu(rich_menu_id)


In [ ]:
@handler.add(MessageEvent, message=TextMessage)
def handle_line_text(event):
    response = chat_with_llm(event.source.user_id, event.message.text)
    line_bot_api.reply_message(event.reply_token, messages=[TextSendMessage(response)])

def chat_with_llm(user_id, input_text):
    user_profile = line_bot_api.get_profile(user_id)
    prmpts = "请假装你是一名专业的心理咨商师，根据下列发问者所提供之问题描述一步步进行思考，并以第一人称口吻给出答复。\n问题描述："

    cc = opencc.OpenCC('tw2s')
    converted_input = cc.convert(input_text)

    cc = opencc.OpenCC('s2tw')
    response, history = model.chat(tokenizer, prmpts + converted_input, history=[], repetition_penalty=1.2)
    response = '\n'.join(response.replace(". ", ".").split()).replace(",", "，").replace(".", ". ")
    converted_response = cc.convert(response)
    field_names = ["user_id", "display_name", "picture_url", "input_text", "response"]

    # 將用戶資料以及聊天紀錄存至users.csv
    with open("/content/drive/MyDrive/users.csv", "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)

        if csvfile.tell() == 0:
            writer.writeheader()

        writer.writerow({
            "user_id": user_profile.user_id,
            "display_name": user_profile.display_name,
            "picture_url": user_profile.picture_url,
            "input_text": input_text,
            "response": converted_response
        })
    return converted_response

In [ ]:
# 主程序運行
port = 5000
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
app.run()

 * ngrok tunnel "https://062f-35-185-113-89.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Aug/2023 11:00:07] "POST / HTTP/1.1" 200 -
<ipython-input-6-96f5097cc38b>:8: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(
INFO:werkzeug:127.0.0.1 - - [12/Aug/2023 11:00:09] "POST / HTTP/1.1" 200 -
<ipython-input-60-243debe48216>:76: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for mo